In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-28 19:39:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.96MB/s    in 0.2s    

2021-10-28 19:39:21 (4.96 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(100)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [6]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|                Y|
|R1LRYU1V0T3O38|          4|            0|          0|   N|                Y|
| R44QKV6FE5CJ2|          5|            0|          0|   N|                Y|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|                Y|
|R1JEEW4C6R89BA|          5|            0|          0|   N|     

In [7]:
# Filter by price on certain columns
vine_df.filter("total_votes=20").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"]).show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R17V95WYC9ZXDD|          1|            0|         20|   N|                Y|
|R3GOWP6NVZ9SYO|          1|            0|         20|   N|                Y|
|R1R68MH1B0U138|          1|            6|         20|   N|                N|
|R18DRZUXWZBQ0M|          1|            7|         20|   N|                Y|
|R1E7D5WADUGQAF|          1|            7|         20|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [8]:
helpful_filter_df = vine_df.filter("helpful_votes/total_votes >= .50")
helpful_filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1IMWXLSA5GWZU|          3|            1|          1|   N|                Y|
|R3AYRT4NAMQSZB|          5|            1|          1|   N|                N|
|R2642823GCWLND|          3|            1|          1|   N|                Y|
|R2AOPNDW6ZWIRN|          5|            1|          2|   N|                Y|
|R2IWG7VDOCXK5U|          5|            1|          1|   N|                Y|
|R34Y0NX6RHTWH1|          5|            1|          2|   N|                Y|
| RDHZ0IP0QWF3G|          1|            1|          1|   N|                Y|
|R115MHJTW7LRMX|          5|            1|          1|   N|                Y|
|R2GFH6IGSW33MP|          4|            1|          1|   N|     

In [9]:
vine_paid_df = helpful_filter_df.filter(helpful_filter_df["vine"] == "Y")
vine_paid_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [10]:
vine_paid_review_count = vine_paid_df.count()
print(vine_paid_review_count)

0


In [11]:
vine_unpaid_df = helpful_filter_df.filter(helpful_filter_df["vine"] == "N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1IMWXLSA5GWZU|          3|            1|          1|   N|                Y|
|R3AYRT4NAMQSZB|          5|            1|          1|   N|                N|
|R2642823GCWLND|          3|            1|          1|   N|                Y|
|R2AOPNDW6ZWIRN|          5|            1|          2|   N|                Y|
|R2IWG7VDOCXK5U|          5|            1|          1|   N|                Y|
|R34Y0NX6RHTWH1|          5|            1|          2|   N|                Y|
| RDHZ0IP0QWF3G|          1|            1|          1|   N|                Y|
|R115MHJTW7LRMX|          5|            1|          1|   N|                Y|
|R2GFH6IGSW33MP|          4|            1|          1|   N|     

In [12]:
vine_unpaid_review_count = vine_unpaid_df.count()
print(vine_unpaid_review_count)

36826


In [18]:
vine_unpaid_5star = vine_unpaid_df.filter(vine_unpaid_df["star_rating"] == "5").count()
print(vine_unpaid_5star)


15289


In [29]:
vine_paid_5star = vine_paid_df.filter(vine_paid_df["star_rating"] == "5").count()
print(vine_paid_5star)

0


In [26]:
vine_unpaid_5star_percent = (vine_unpaid_5star / vine_unpaid_review_count)
print(vine_unpaid_5star_percent)

0.4151686308586325


In [37]:
total_review_count = (vine_paid_review_count + vine_unpaid_review_count)
print(total_review_count)

36826


In [27]:
vine_paid_5star_percent = (vine_paid_5star / vine_paid_review_count)
print(vine_paid_5star_percent)

TypeError: ignored

In [40]:
print("Total UNPAID reviews: %f" % vine_unpaid_review_count)
print("Total UNPAID 5 star reviews: %f" % vine_unpaid_5star)
print("Percent UNPAID 5 star reviews: %f" % vine_unpaid_5star_percent)
print("Total PAID reviews: %f" % vine_paid_review_count) 
print("Total PAID 5 star reviews: %f" % vine_paid_5star)
print("Percent PAID 5 star reviews is an error, as you cannot divide by zero")

Total UNPAID reviews: 36826.000000
Total UNPAID 5 star reviews: 15289.000000
Percent UNPAID 5 star reviews: 0.415169
Total PAID reviews: 0.000000
Total PAID 5 star reviews: 0.000000
Percent PAID 5 star reviews is an error, as you cannot divide by zero
